In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
%config Completer.use_jedi = False


In [3]:
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [4]:
lk_params=dict(winSize=(200,200), maxLevel = 2,
              criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [7]:
cap=cv2.VideoCapture(0)

ret,prev_frame=cap.read()

prev_gray=cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

prev_points=cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask=np.zeros_like(prev_frame)

while True:
    
    ret,frame=cap.read()
    
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    nextPts,status,err=cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prev_points,None,**lk_params)
    
    good_new=nextPts[status==1]
    good_prev=prev_points[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prev_points = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

In [8]:
good_new

array([[397.08356, 289.04263],
       [262.18256, 354.00085],
       [251.1873 , 320.99905],
       [293.1745 , 295.97803],
       [303.17346, 296.99542],
       [240.19221, 280.96356],
       [363.1065 , 296.00403],
       [242.18938, 289.96378],
       [390.20346, 416.0059 ],
       [353.19476, 401.00876]], dtype=float32)

In [11]:
cap=cv2.VideoCapture(0)
ret,prev_frame=cap.read()
prev_gray=cv2.cvtColor(prev_frame,cv2.COLOR_RGB2GRAY)
prev_points=cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)
mask=np.zeros_like(prev_frame)
while True:
    ret,frame=cap.read()
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
    nxt,status,err=cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prev_points,None,**lk_params)
    good_new=nxt[status==1]
    good_prev=prev_points[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new=new.ravel()
        x_prev,y_prev=prev.ravel()
        
        mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        frame=cv2.circle(frame,(x_new,y_new),8, (0,0,255), -1)
        
    img=cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prev_points = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

In [6]:
%config Completer.use_jedi = False


In [ ]:
import cv2
import numpy as np

def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()


    return tracker
 
tracker=ask_for_tracker()
tracker_name=str(tracker).split()[0][1:]



# Read video
cap = cv2.VideoCapture(0)

# Read first frame.
ret, frame = cap.read()

roi=cv2.selectROI(frame,False)

ret=tracker.init(frame,roi)

while True:
    
    ret, frame = cap.read()
        
    success,roi=tracker.update(frame)
    
    (x,y,w,h)=tuple(map(int,roi))
    
    if success:
        pt1=(x,y)
        pt2=(x+w,y+h)
        
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
    else:
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)
        
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()
    

In [3]:
i=0
while i<5:
    print(i)
    i=i+1
    if i==3:
        break
else:
    print(0)

0
1
2


In [12]:
background=None
accumulated_weight=0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

def calc_accumulated_weight(frame,accumulated_weight):
    global background
    
    if background is None:
        background=frame.copy().astype('float')
        return None
    cv2.accumulated_weight(frame,background,accumulated_weight)
    

def segment(frame,threshold_min=25):
    diff=cv2.absdiff(background.astype('uint8'),frame)
    ret,thresholded=cv2.threshold(diff,threshold_min,cv2.THRESH_BINARY)
    
    image,contours,hierarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours)==0:
        return None
    else:
        hand_segment=max(contours,key=cv2.contourArea)
        
    return (thresholded,hand_segment)


def count_fingers(thresholded,hand_segment):
    conv_hull=cv2.convexHull(hand_segment)
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    distance=pairwise.euclidean_distances((cX,cY),(top,bottom,left,rigth))
    
    max_distance=distance.max()
    radius = int(0.9*max_distance)
    circumfrence = (2*np.pi*radius)
    
    circular_roi=np.zeros(thresholded[:2])
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count=0
    for cnt in contours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

cap=cv2.VideoCapture(0)
num_frames=0

while True:
    ret,frame=cap.read()
    frame_copy=frame.copy()
    roi=frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accumulated_weight(gray,accumulated_weight)
        if num_frames<=59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        hand=segment(gray)
        
        if hand is not None:
            thresholded , hand_segment = hand
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],1,(255,0,0),5)
            fingers = count_fingers(thresholded,hand_segment)
            
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded',thresholded)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    num_frames += 1
    
    cv2.imshow('Finger Count',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
             
        

AttributeError: module 'cv2' has no attribute 'accumulated_weight'

In [11]:
from sklearn.metrics import pairwise